<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/actual_words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
#!pip install tokenizers

In [35]:
!pip install transformers
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.0 MB/s eta 0:00:00


In [3]:
#from tokenizers import Tokenizer
#from tokenizers.models import BPE
#tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

In [4]:
#from tokenizers.trainers import BpeTrainer
#trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

In [5]:
#from tokenizers.pre_tokenizers import Whitespace
#tokenizer.pre_tokenizer = Whitespace()

In [6]:
#!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
#!unzip wikitext-103-raw-v1.zip

--2023-10-04 16:06:05--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.121.24, 52.217.71.238, 16.182.105.8, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.121.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191984949 (183M) [application/zip]
Saving to: ‘wikitext-103-raw-v1.zip.1’

wikitext-103-raw-v1 100%[===================>] 183.09M  41.1MB/s    in 5.0s    

2023-10-04 16:06:10 (36.4 MB/s) - ‘wikitext-103-raw-v1.zip.1’ saved [191984949/191984949]

Archive:  wikitext-103-raw-v1.zip
replace wikitext-103-raw/wiki.test.raw? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [7]:
#files = [f"wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
#tokenizer.train(files, trainer)

In [8]:
#tokenizer.save("tokenizer-wiki.json")

In [9]:
#tokenizer = Tokenizer.from_file("tokenizer-wiki.json")

In [37]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")
out_tensor = torch.tensor(output)

In [38]:
out_tensor

tensor([ 101, 8667,  117,  194,  112, 1155,  106, 1731, 1132, 1128,  100,  136,
         102])

In [40]:
tokenizer.decode(out_tensor)

"[CLS] Hello, y'all! How are you [UNK]? [SEP]"

In [13]:
import requests
import io

# Get the content of the page
response = requests.get("https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt")

# Write the content to a file
with io.open('traintext.txt', 'w', encoding='utf-8') as f:
    f.write(response.text)

In [14]:
with open("traintext.txt", 'r') as f:
  text = f.readlines()

In [15]:
text[:10]

['First Citizen:\n',
 'Before we proceed any further, hear me speak.\n',
 '\n',
 'All:\n',
 'Speak, speak.\n',
 '\n',
 'First Citizen:\n',
 'You are all resolved rather to die than to famish?\n',
 '\n',
 'All:\n']

In [16]:
import random

In [17]:
text_len = len(text) - 1

In [42]:
context_len = 20
emb_dim = 10
#tokenizer.enable_truncation(context_len)
#tokenizer.enable_padding(direction='left', length=context_len)

In [19]:
#vocab_size = tokenizer.get_vocab_size()

pretokenize? aaaa but it takes so long

In [20]:
#text = torch.tensor([tokenizer.encode(t).ids for t in text])

In [21]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [50]:
def make_batch(batch):
  xs = []
  ys = []
  for sentence in batch:
    context = ""
    for word, next in zip(sentence, sentence[1:]):
      context += word
      xs.append(tokenizer.encode(context, padding=True, max_length=20))
      ys.append(torch.argmax(torch.tensor(tokenizer.encode(word, padding=True, max_length=20))))
  return torch.tensor(xs).to(device), torch.tensor(ys).to(device)

In [44]:
C = torch.randn((vocab_size, emb_dim)).to(device)
w1 = torch.randn((emb_dim * context_len, 100)).to(device) * 0.1
b1 = torch.randn(100).to(device) * 0.01
w2 = torch.randn((100, vocab_size)).to(device) * 0.1
b2 = torch.randn(vocab_size).to(device) * 0.01

params = [C, w1, b1, w2, b2]
for p in params:
  p.requires_grad = True

In [45]:
import torch.nn.functional as F

In [46]:
num_epochs = 2000
batch_size = 32
lr = 0.1

In [51]:
for epoch in range(num_epochs):
  batch = [text[random.randint(0, text_len)] for i in range(batch_size)]
  #words = torch.tensor([tokenizer.encode_batch(batch)[i].ids for i in range(batch_size)])
  x, y = make_batch(batch)

  emb = C[x].view(-1, emb_dim * context_len)
  l1 = (emb @ w1 + b1).tanh()
  out = (l1 @ w2 + b2).tanh()
  loss = F.cross_entropy(out, y)

  for p in params:
    p.grad = None

  loss.backward()
  for p in params:
    p.data -= p.grad * lr
  if epoch % 100 == 0:
    print(loss.item())

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2640: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


ValueError: ignored

In [27]:
@torch.no_grad()
def forward(text):
  ins = text
  x = torch.tensor(tokenizer.encode(text).ids)
  emb = C[x].view(-1, emb_dim * context_len)
  l1 = (emb @ w1 + b1).tanh()
  out = (l1 @ w2 + b2).tanh()
  out = out.exp()
  out /= out.sum(1, keepdim=True)
  sampled = torch.multinomial(out, num_samples=1)
  return sampled

In [28]:
import matplotlib.pyplot as plt

In [29]:
lol = forward('hi')
print(lol)
tokenizer.decode_batch(lol.cpu().numpy())

tensor([[2255]], device='cuda:0')


['僕']

In [34]:
def cont(text, max_len=20):
  context = text
  for x in range(max_len):
    context += tokenizer.decode_batch(forward(context).cpu().numpy())[0]
  return context

cont("Romeo")

'Romeoasserted須諭JustintingscanalscathconsumedcoupsuperiormodifiedAntarcticrefereeventuallyOlympventionsenergetic方Clements'

In [31]:
tokenizer.encode("[UNK]").ids

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [32]:
tokenizer.get_vocab_size()

30000

5011? y doe

why did it learn chinese? It's really consistently chinese too. Maybe I need a smaller vocab size?